In [1]:
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn import linear_model
import nltk
import random
import re
import nltk.corpus
import sklearn
import numpy
import csv
from sklearn import metrics
import sys
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english", ignore_stopwords=True) #se não ignorar os stopwords, não vai remover eles depois

#inicializando bag de stopwords
stopwords = nltk.corpus.stopwords.words('english')

import os
print(os.listdir("../input"))

['cleandata', 'jigsaw-unintended-bias-in-toxicity-classification']


In [2]:
base = []
csvFile = "../input/cleandata/cleanDataTrain.csv"

baseTrain = []
labels = []
allWords = []
with open(csvFile) as csvfile:
    import codecs
    ifile = open(csvFile, "rb")
    read = csv.reader(codecs.iterdecode(ifile, 'utf-8'))

    for row in read:
        try:
            temp1 = row[1]
            allWords.append(temp1)
            baseTrain.append(temp1)
            temp2 = float(row[0])
            labels.append(temp2)
        except IndexError:
            pass

In [3]:
baseTest = []
csvFile = "../input/cleandata/cleanDataTest.csv"
ids = []
with open(csvFile) as csvfile:
    import codecs
    ifile = open(csvFile, "rb")
    read = csv.reader(codecs.iterdecode(ifile, 'utf-8'))

    for row in read:
        try:
            # Se for pegar o valor float
            temp2 = row[0]
            temp1 = row[1]
            if(temp2!="id"):
                baseTest.append(temp1)
                ids.append(temp2)
        except IndexError:
            pass

In [4]:
x = baseTrain
y = labels

In [5]:
x_min = []
y_min = []
for u in range(len(y)):
    if(y[u]!=0):
        x_min.append(x[u])
        y_min.append(y[u])

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
#vectorizer_TF = CountVectorizer()
#X_TFIDF = vectorizer_TF.fit_transform(x_min)
#X_TestTFIDF = vectorizer_TF.transform(baseTest)

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer_TFIDF = TfidfVectorizer()
X_TFIDF = vectorizer_TFIDF.fit_transform(x_min)
X_TestTFIDF = vectorizer_TFIDF.transform(baseTest)

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_TFIDF, y_min, test_size=0.10, random_state=42)

In [17]:
import torch
from torch.utils.data import TensorDataset, DataLoader

Acoo_trainX = X_train.tocoo()
Apt_trainX = torch.sparse.LongTensor(torch.LongTensor([Acoo_trainX.row.tolist(), Acoo_trainX.col.tolist()]),
                              torch.LongTensor(Acoo_trainX.data.astype(numpy.int32)))
Acoo_testX = X_test.tocoo()
Apt_testX = torch.sparse.LongTensor(torch.LongTensor([Acoo_testX.row.tolist(), Acoo_testX.col.tolist()]),
                              torch.LongTensor(Acoo_testX.data.astype(numpy.int32)))
Acoo_trainY = numpy.asarray(y_train)
Apt_trainY = torch.from_numpy(Acoo_trainY)

Acoo_testY = numpy.asarray(y_test)
Apt_testY = torch.from_numpy(Acoo_testY)

# create Tensor datasets
train_data = TensorDataset(Apt_trainX, Apt_trainY)
test_data = TensorDataset(Apt_testX, Apt_testY)

# dataloaders
batch_size = 50

# shuffling and batching data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [18]:
# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

Training on GPU.


In [19]:

import torch.nn as nn
import torch.nn.functional as F

class SentimentCNN(nn.Module):
    """
    The embedding layer + CNN model that will be used to perform sentiment analysis.
    """

    def __init__(self, embed_model, vocab_size, output_size, embedding_dim,
                 num_filters=100, kernel_sizes=[3, 4, 5], freeze_embeddings=True, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super(SentimentCNN, self).__init__()

        # set class vars
        self.num_filters = num_filters
        self.embedding_dim = embedding_dim
        
        # 1. embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        # set weights to pre-trained
        self.embedding.weight = nn.Parameter(torch.from_numpy(embed_model.vectors)) # all vectors
        # (optional) freeze embedding weights
        if freeze_embeddings:
            self.embedding.requires_grad = False
        
        # 2. convolutional layers
        self.convs_1d = nn.ModuleList([
            nn.Conv2d(1, num_filters, (k, embedding_dim), padding=k-2) 
            for k in kernel_sizes])
        
        # 3. final, fully-connected layer for classification
        self.fc = nn.Linear(len(kernel_sizes) * num_filters, output_size) 
        
        # 4. dropout and sigmoid layers
        self.dropout = nn.Dropout(drop_prob)
        self.sig = nn.Sigmoid()
        
    
    def conv_and_pool(self, x, conv):
        """
        Convolutional + max pooling layer
        """
        # squeeze last dim to get size: (batch_size, num_filters, conv_seq_length)
        # conv_seq_length will be ~ 200
        x = F.relu(conv(x)).squeeze(3)
        
        # 1D pool over conv_seq_length
        # squeeze to get size: (batch_size, num_filters)
        x_max = F.max_pool1d(x, x.size(2)).squeeze(2)
        return x_max

    def forward(self, x):
        """
        Defines how a batch of inputs, x, passes through the model layers.
        Returns a single, sigmoid-activated class score as output.
        """
        # embedded vectors
        embeds = self.embedding(x) # (batch_size, seq_length, embedding_dim)
        # embeds.unsqueeze(1) creates a channel dimension that conv layers expect
        embeds = embeds.unsqueeze(1)
        
        # get output of each conv-pool layer
        conv_results = [self.conv_and_pool(embeds, conv) for conv in self.convs_1d]
        
        # concatenate results and add dropout
        x = torch.cat(conv_results, 1)
        x = self.dropout(x)
        
        # final logit
        logit = self.fc(x) 
        
        # sigmoid-activated --> a class score
        return self.sig(logit)

In [23]:
X_TFIDF.shape[1]

109767

In [29]:
# Instantiate the model w/ hyperparams

vocab_size = X_TFIDF.shape[1]
embed_lookup = vectorizer_TFIDF
output_size = 1 # binary class (1 or 0)
embedding_dim = vocab_size # 300-dim vectors
num_filters = 100
kernel_sizes = [3, 4, 5]

net = SentimentCNN(embed_lookup, vocab_size, output_size, embedding_dim,
                   num_filters, kernel_sizes)

print(net)

RuntimeError: [enforce fail at CPUAllocator.cpp:56] posix_memalign(&data, gAlignment, nbytes) == 0. 12 vs 0


In [ ]:
from sklearn import svm
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
#clf = svm.SVR()
#clf = linear_model.LinearRegression()
#clf = DecisionTreeRegressor(max_depth=10)
clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr')
#clf = GaussianNB()

#Classificação com probabilidade
y_train_class = []
for t in range(len(y_train)):
    if(y_train[t]<0.5):
        y_train_class.append(0)
    else:
        y_train_class.append(1)

clf.fit(X_train,y_train_class)

In [ ]:
from sklearn import metrics
y_pred = clf.predict(X_test) 
y_pred_prob = clf.predict_proba(X_test) 
print(y_pred_prob)
print(X_test.shape)

y_pred_class = y_pred
y_test_class = []
y_pred_prob_tox = []


for t in range(len(y_test)):
    if(y_test[t]<0.5):
        y_test_class.append(0)
    else:
        y_test_class.append(1)
    y_pred_prob_tox.append(y_pred_prob[t][0])

mse = metrics.mean_squared_error(y_test, y_pred_prob_tox)
auc = metrics.roc_auc_score(y_test_class, y_pred_class)
acc = metrics.accuracy_score(y_test_class, y_pred_class)
prec = metrics.precision_score(y_test_class, y_pred_class)
rec = metrics.recall_score(y_test_class, y_pred_class)

print(auc)
print(acc)
print(prec)
print(rec)
print(mse)

with open('results_validation.csv', mode='w') as employee_file:
    writer = csv.writer(employee_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(["Logistic Regression"])
    writer.writerow(["mse: ", mse])
    writer.writerow(["auc: ", auc])
    writer.writerow(["accuracy: ", acc])
    writer.writerow(["precision: ", prec])
    writer.writerow(["recall: ", rec])

In [ ]:
# from sklearn import metrics
# y_pred = clf.predict(X_test) 

# y_pred_class = []
# y_test_class = []

# for t in range(len(y_test)):
#     if(y_test[t]<0.5):
#         y_test_class.append(0)
#     else:
#         y_test_class.append(1)
#     if(y_pred[t]<0.5):
#         y_pred_class.append(0)
#     else:
#         y_pred_class.append(1)

# mse = metrics.mean_squared_error(y_test, y_pred)
# auc = metrics.roc_auc_score(y_test_class, y_pred_class)
# acc = metrics.accuracy_score(y_test_class, y_pred_class)
# prec = metrics.precision_score(y_test_class, y_pred_class)
# rec = metrics.recall_score(y_test_class, y_pred_class)

# print(auc)
# print(mse)
# print(acc)
# print(prec)
# print(rec)

# with open('results_validation.csv', mode='w') as employee_file:
#     writer = csv.writer(employee_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#     writer.writerow(["Linear Regression"])
#     writer.writerow(["mse: ", mse])
#     writer.writerow(["auc: ", auc])
#     writer.writerow(["accuracy: ", acc])
#     writer.writerow(["precision: ", prec])
#     writer.writerow(["recall: ", rec])

In [ ]:
y_pred_test = clf.predict_proba(X_TestTFIDF)
with open('submission.csv', mode='w') as employee_file:
    
    writer = csv.writer(employee_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(["id", "prediction"])
    for id in range(len(ids)):
        print(ids[id])
        print(y_pred_test[id][1])
        writer.writerow([ids[id], y_pred_test[id][1]])